In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data


#### Adjust pandas Outcput

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [4]:
%%time
canary_p = '../../temp/CANARY.p'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 26.4 s, sys: 9.6 s, total: 36 s
Wall time: 19.3 s


In [5]:
pmpm.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2570122               95      40392        0             0                  1       0          1                    0             0          0              1          0           0       1                 0            0                  0                  1                     0        1                   0              1                        0                  0            0                0         12                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2570122               94      19837        0             0                  0       0          1                    0             1          0              0          0           0       1                 0            0                  0                  0                     0        1                   0              0                        0                  0            0                0          7                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR MBR_GNDR     MYR  
0            0                   0        0                 0                       1               0                0          0           0                   0                 0        M  202002  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0        M  201810

#### An inspection of categories registered in biometrics data

In [6]:
%%time
pd.DataFrame(biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)).T

CPU times: user 19.8 s, sys: 5.09 s, total: 24.9 s
Wall time: 24.9 s


BIOMETRIC_CATEGORY     A1C       BMI    BNP  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  DCSI-SERUM  Ferritin    Height  Hematocrit  Hemoglobin    Iron  NT_proBNP     PSA  Potassium  Troponin    Weight    eGFR
0                   790853  10034183  42552                     4902004                    5238991            1305880     628758              81297     2734084    219613  11808045     2574412     2919909  197878       9258  310283    2666981     94030  14000267  907656

## Steps in creating causal data
``` python
%%time
from data_functions import  biometrics_f, pmpm_f, canary_f, merge_filter_separate, control_outcomes, treated_outcomes, combine_treated_controls
biometrics = biometrics_f(biometrics, 'BMI', True)
pmpm = pmpm_f(pmpm, True, 12)
canary = canary_f(canary)
merged, treated, controls, canary = merge_filter_separate(biometrics, pmpm, canary, merge_how = 'inner')
controls_final = control_outcomes(controls, max_days_apart = 180, min_days_apart = 120)
treated_final, treated_help = treated_outcomes(treated, max_days_apart = 180, min_days_apart = 120)
df = combine_treated_controls(treated_final, control_finals)
```

## Create dataset

In [7]:
%%time
df, treated_help = create_causal_data(canary.copy(), pmpm.copy(), biometrics.copy(), outcome_under_study = 'BMI', drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = "inner",
                      max_days_apart = 180, min_days_apart = 120)

/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls['OUTCOME'] = outcomes
/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls['OUTCOME2'] = outcome2


CPU times: user 1min 54s, sys: 16.8 s, total: 2min 11s
Wall time: 2min 11s


In [8]:
df.head(3)

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0               81        414                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          2                      0            0                0            0                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1               82        101                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
2               78      13007                BMI        0             0                  0       0          0                    0             1          0              0          0           1       0                 0            1                  0                  0                     0        0                   0              1                        0                  0            0                0          8                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             1         0               0            0                   0                      0                      0           0                0              0   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CA

In [9]:
df.TREATED.value_counts()

0    112345
1       371
Name: TREATED, dtype: int64

In [10]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle('../../matching_data/data.p')
df.shape

(112716, 78)

### <span style="color:red">Attention </span>
biometrics have datapoints from 1920, 1940... and one datapoint from the future 2179


### DoWhy demonstration on the data(install dowhy with)
```pip3 install dowhy --user```

In [17]:
import dowhy
from dowhy import CausalModel

import logging
logging.getLogger("dowhy").setLevel(logging.WARNING)
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Specify Common Causes Names

In [40]:
common_causes = np.setdiff1d(df.columns, ['BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 
                                          'PERS_ID', 'RESULT_VALUE', 'TREATED'])

df.TREATED = df.TREATED.astype(bool)


### Create data dictionary for causal modeling

In [42]:
data = {'df':df, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME1',
        'treatment_name':'TREATED'}

# 1) Define Causal Model

In [43]:
model = CausalModel(
            data=data['df'],
            treatment=data['treatment_name'],
            outcome=data['outcome1'],
            common_causes=data['common_causes_names'],
            )

# 2) Check if the Effect if Identifiable

In [44]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
#print(identified_estimand)

# 3) Use an estimation method (ex: propensity score matching)

In [49]:
propensity_match_estimate = model.estimate_effect(identified_estimand,
                            method_name="backdoor.propensity_score_stratification",
                            test_significance = False,
                            confidence_intervals = False,
                            target_units="att")

#print(propensity_match_estimate)

/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Get propensity scores for each subject to use accordingly later

In [51]:
propensity_match_estimate.propensity_scores

0         0.003472
1         0.003650
2         0.000384
3         0.003167
4         0.000845
            ...   
112711    0.001595
112712    0.003240
112713    0.001660
112714    0.000290
112715    0.001744
Name: propensity_score, Length: 112716, dtype: float64

All necessary parameters from the model can be found with:
```python
propensity_match_estimate.params
```


## For each data point all the parameters are inserted into the dataframe automatically

In [53]:
df.head()

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0               81        414                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          2                      0            0                0            0                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1               82        101                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
2               78      13007                BMI        0             0                  0       0          0                    0             1          0              0          0           1       0                 0            1                  0                  0                     0        0                   0              1                        0                  0            0                0          8                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             1         0               0            0                   0                      0                      0           0                0              0   
3               76        898                BM